In [1]:
import time, warnings
import pandas as pd
import numpy as np
import rasterio
import os
from joblib import parallel_backend

from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/Users/jonny.sanchez/Documents/tesis/estaciones_final_maestria.csv',sep=";")
df.head()

,fuente,ciudad,codigo_estacion,fecha_toma,anio,mes,dia,medicion,sea_water,fresh_water,...,clouds,bare_ground,veg,lst,ndbi,ndvi,ndwi,st_emissivity,norte,este
0,dimar,barranquilla,0000000004,2020-03-29 07:00:00.000,2020,3,29,26.40,0.04,0.88,...,0.0,0.00,0.0,29.455811,-0.011720,-0.016581,0.002060,0.9880,2.785929e+06,4.798071e+06
1,dimar,barranquilla,0000000004,2021-02-12 13:00:00.000,2021,2,12,28.12,0.08,0.72,...,0.0,0.00,0.0,28.509033,-0.013787,0.060807,-0.105339,0.9880,2.785929e+06,4.798071e+06
2,dimar,barranquilla,0000000004,2022-02-23 13:00:00.000,2022,2,23,27.81,0.25,0.70,...,0.0,0.00,0.0,29.517365,-0.096698,0.052519,-0.084309,0.9880,2.785929e+06,4.798071e+06
3,dimar,barranquilla,0CP03FL033,2020-03-29 13:00:00.000,2020,3,29,28.50,0.00,0.28,...,0.0,0.04,0.0,36.818237,0.073587,0.277937,-0.362358,0.9669,2.778642e+06,4.801240e+06
4,dimar,barranquilla,0CP03FL033,2021-02-12 13:00:00.000,2021,2,12,28.60,0.00,0.32,...,0.0,0.20,0.0,34.975922,0.049616,0.316371,-0.368710,0.9669,2.778642e+06,4.801240e+06


In [3]:
y = df['medicion'].values

bands = ['anio','mes','dia','sea_water','fresh_water','builds','clouds','bare_ground','veg','lst','ndbi','norte','este']
available_cols = [c for c in bands if c in df.columns]

X = df[available_cols]

In [4]:
X.corr()

,anio,mes,dia,sea_water,fresh_water,builds,clouds,bare_ground,veg,lst,ndbi,norte,este
anio,1.000000,-0.351678,0.569311,0.024810,0.064244,-0.099667,NaN,0.042142,0.021357,-0.107609,-0.032938,0.011752,-0.001594
mes,-0.351678,1.000000,-0.159761,-0.008129,-0.010227,0.087525,NaN,0.115957,-0.144120,0.162710,0.013589,0.020809,0.014434
dia,0.569311,-0.159761,1.000000,0.009942,0.119846,-0.028071,NaN,0.034618,-0.044664,-0.040819,-0.076768,0.020673,0.005818
sea_water,0.024810,-0.008129,0.009942,1.000000,-0.121512,-0.526786,NaN,-0.356849,-0.503944,-0.743150,0.090142,0.085052,0.016479
fresh_water,0.064244,-0.010227,0.119846,-0.121512,1.000000,-0.057815,NaN,-0.066334,-0.070632,-0.096071,-0.016298,0.021524,-0.031713
builds,-0.099667,0.087525,-0.028071,-0.526786,-0.057815,1.000000,NaN,-0.042686,-0.277986,0.756813,-0.043944,-0.128136,-0.198516
clouds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bare_ground,0.042142,0.115957,0.034618,-0.356849,-0.066334,-0.042686,NaN,1.000000,-0.082940,0.419740,-0.029842,0.001348,-0.004891
veg,0.021357,-0.144120,-0.044664,-0.503944,-0.070632,-0.277986,NaN,-0.082940,1.000000,-0.024201,-0.048643,0.010942,0.190711
lst,-0.107609,0.162710,-0.040819,-0.743150,-0.096071,0.756813,NaN,0.419740,-0.024201,1.000000,-0.054163,-0.286621,-0.251972


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'LinearRegression' : (
        Pipeline([("scaler", StandardScaler()),("model",LinearRegression())]), 
        {}
    ),
    'RandomForestRegressor': (
        Pipeline([("model", RandomForestRegressor(random_state=42,n_jobs=-1))]),
        {"model__n_estimators":[200], 
         "model__max_depth":[None,8],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]}
    ),
    'ExtraTrees':(
        Pipeline([("model",ExtraTreesRegressor(random_state=42,n_jobs=-1))]),
        {
        "model__n_estimators": [300, 500],
        "model__max_depth": [None, 15],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]}
    ),
    'GradientBoostingRegressor': (
        Pipeline([("model", GradientBoostingRegressor(random_state=42))]),
        {"model__n_estimators":[200,400], 
         "model__learning_rate":[0.05,0.1], 
         "model__max_depth":[2,3]}
    ),
    'SVR': (
        Pipeline([("scaler", StandardScaler()), ("model", SVR())]),
        {"model__C": [1.0, 10.0], 
         "model__epsilon": [0.1, 0.3], 
         "model__kernel": ["rbf"]}
    ),
    'MLPRegressor': (
        Pipeline([("scaler", StandardScaler()), ("model", MLPRegressor(random_state=42, max_iter=3000))]),
        {"model__hidden_layer_sizes": [(64, 32), (128, 64, 32)], 
         "model__alpha": [1e-5, 1e-4, 1e-3, 1e-2], 
         "model__activation": ["relu","tanh"],
         "model__solver": ["adam", "lbfgs"],}
    ),
}

In [6]:
k          = 10
cv         = KFold(n_splits=k, shuffle=True, random_state=42)

results = []
best_models = {}

for name, (pipe, grid) in models.items():
    gs = GridSearchCV(pipe, grid, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=1, refit=True)
    tic = time.perf_counter()
    gs.fit(X_train, y_train)
    fit_time = round(time.perf_counter() - tic, 3)

    y_pred = gs.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = float(np.sqrt(mse))
    mae = float(mean_absolute_error(y_test, y_pred))
    r2 = float(r2_score(y_test, y_pred))

    with parallel_backend("threading"):
        tic = time.perf_counter()
        y_pred_oof = cross_val_predict(gs.best_estimator_, X, y, cv=cv, n_jobs=-1)
        cv_pred_time = round(time.perf_counter() - tic, 3)

    rmse_oof = float(np.sqrt(mean_squared_error(y, y_pred_oof)))
    mae_oof  = float(mean_absolute_error(y, y_pred_oof))
    r2_oof   = float(r2_score(y, y_pred_oof))

    print(f"\n=== {name} ===")
    print("Mejores params CV: ", gs.best_params_)
    print(f"Tiempo entrenamiento: {fit_time}s | Tiempo OOF-predict: {cv_pred_time}s")
    print(f"[TEST]   RMSE={rmse:.3f}  MAE={mae:.3f}  R2={r2:.3f}")
    print(f"[OOF-CV] RMSE={rmse_oof:.3f}  MAE={mae_oof:.3f}  R2={r2_oof:.3f}")

    results.append({
        "modelo": name,
        "best_params": gs.best_params_,
        "fit_time_s": fit_time,
        "cv_pred_time_s": cv_pred_time,
        "test_RMSE": rmse, "test_MAE": mae, "test_R2": r2,
        "oof_RMSE": rmse_oof, "oof_MAE": mae_oof, "oof_R2": r2_oof,
        "cv_best_RMSE": float(-gs.best_score_),
    })

    best_models[name] = gs.best_estimator_

res_df = pd.DataFrame(results).sort_values("test_RMSE", ascending=False)
display(res_df)


=== LinearRegression ===
Mejores params CV:  {}
Tiempo entrenamiento: 0.018s | Tiempo OOF-predict: 0.037s
[TEST]   RMSE=2.662  MAE=1.237  R2=-0.012
[OOF-CV] RMSE=1.658  MAE=0.972  R2=0.555

=== RandomForestRegressor ===
Mejores params CV:  {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Tiempo entrenamiento: 6.588s | Tiempo OOF-predict: 0.783s
[TEST]   RMSE=0.793  MAE=0.536  R2=0.910
[OOF-CV] RMSE=0.675  MAE=0.454  R2=0.926

=== ExtraTrees ===
Mejores params CV:  {'model__max_depth': 15, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 500}
Tiempo entrenamiento: 16.491s | Tiempo OOF-predict: 1.238s
[TEST]   RMSE=0.745  MAE=0.522  R2=0.921
[OOF-CV] RMSE=0.600  MAE=0.420  R2=0.942

=== GradientBoostingRegressor ===
Mejores params CV:  {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 200}
Tiempo entrenamiento: 4.249s | Tiempo OOF-predict: 0.26s
[TEST]   RMSE=0

,modelo,best_params,fit_time_s,cv_pred_time_s,test_RMSE,test_MAE,test_R2,oof_RMSE,oof_MAE,oof_R2,cv_best_RMSE
0,LinearRegression,{},0.018,0.037,2.661928,1.236742,-0.012239,1.658309,0.972124,0.554636,1.178727
5,MLPRegressor,"{'model__activation': 'tanh', 'model__alpha': ...",179.993,23.473,1.378685,0.805781,0.728468,0.911594,0.612602,0.865418,0.964654
4,SVR,"{'model__C': 10.0, 'model__epsilon': 0.1, 'mod...",0.075,0.028,1.305656,0.890044,0.756473,1.012137,0.706516,0.834094,0.936149
1,RandomForestRegressor,"{'model__max_depth': None, 'model__min_samples...",6.588,0.783,0.793121,0.536449,0.910139,0.674942,0.453981,0.926224,0.708449
2,ExtraTrees,"{'model__max_depth': 15, 'model__min_samples_l...",16.491,1.238,0.744830,0.522159,0.920749,0.599835,0.419821,0.941730,0.637356
3,GradientBoostingRegressor,"{'model__learning_rate': 0.1, 'model__max_dept...",4.249,0.260,0.676048,0.488248,0.934710,0.617351,0.438786,0.938277,0.670877


In [7]:
feature_order = list(X.columns)
print("feature_order:", feature_order)

feature_order: ['anio', 'mes', 'dia', 'sea_water', 'fresh_water', 'builds', 'clouds', 'bare_ground', 'veg', 'lst', 'ndbi', 'norte', 'este']


In [ ]:
def cargar_banda(path):
    with rasterio.open(path) as src:
        return src.read(), src.profile

def grilla_xy(profile):
    T = profile["transform"]
    W = profile["width"]
    H = profile["height"]
    xs = T.c + np.arange(W) * T.a
    ys = T.f + np.arange(H) * T.e
    X, Y = np.meshgrid(xs, ys)
    return X.astype("float32"), Y.astype("float32")

def read_resample(path, ref_profile):
    with rasterio.open(path) as src:
        if (src.width, src.height) != (ref_profile["width"], ref_profile["height"]) or src.transform != ref_profile["transform"]:
            data = src.read(
                out_shape=(1, ref_profile["height"], ref_profile["width"]),
                resampling=rasterio.enums.Resampling.nearest
            )
            return data.astype("float32")
        else:
            return src.read().astype("float32")

def clasificar_rasters(path_raiz, best_models):
    for folder_name in os.listdir(path_raiz):
        folder_path = os.path.join(path_raiz, folder_name)
        if os.path.isdir(folder_path):
            date = anio = mes = dia = sea_water = fresh_water = builds = clouds = bare_ground = veg = lst = ndbi = ndvi = ndwi = st_emissivity = None
            date = folder_name.split(sep='_')[3]
            anio_file = int(date[0:4])
            mes_file = int(date[4:6])
            dia_file = int(date[6:8])
            for file_name in os.listdir(folder_path):
                if file_name.endswith("c0_sea_water.TIF"):
                    sea_water = os.path.join(folder_path,file_name)
                elif file_name.endswith("c1_fresh_water.TIF"):
                    fresh_water = os.path.join(folder_path,file_name)
                elif file_name.endswith("c2_builds.TIF"):
                    builds = os.path.join(folder_path,file_name)
                elif file_name.endswith("c3_clouds.TIF"):
                    clouds = os.path.join(folder_path,file_name)
                elif file_name.endswith("c4_bare_ground.TIF"):
                    bare_ground = os.path.join(folder_path,file_name)
                elif file_name.endswith("c5_vegetation.TIF"):
                    veg = os.path.join(folder_path,file_name)
                elif file_name.endswith("LST.TIF"):
                    lst = os.path.join(folder_path,file_name)
                elif file_name.endswith("NDBI.TIF"):
                    ndbi = os.path.join(folder_path,file_name)
                elif file_name.endswith("NDVI.TIF"):
                    ndvi = os.path.join(folder_path,file_name)
                elif file_name.endswith("NDWI.TIF"):
                    ndwi = os.path.join(folder_path,file_name)
                elif file_name.endswith("ST_EMIS.TIF"):
                    st_emissivity = os.path.join(folder_path,file_name)

        # cargar y apilar
            sea_water, _ = cargar_banda(sea_water)
            fresh_water, _ = cargar_banda(fresh_water)
            builds, _ = cargar_banda(builds) 
            clouds, _ = cargar_banda(clouds) 
            bare_ground, _ = cargar_banda(bare_ground) 
            veg, _ = cargar_banda(veg) 
            lst, profile = cargar_banda(lst) 
            ndbi, _ = cargar_banda(ndbi) 
            ndvi, _ = cargar_banda(ndvi) 
            ndwi, _ = cargar_banda(ndwi) 
            st_emissivity, _ = cargar_banda(st_emissivity)

            assert sea_water.shape[1:] == fresh_water.shape[1:] == builds.shape[1:] == clouds.shape[1:] == bare_ground.shape[1:] == veg.shape[1:] == lst.shape[1:] == ndbi.shape[1:] == ndwi.shape[1:] == ndvi.shape[1:] == st_emissivity.shape[1:], "Las dimensiones no coinciden."
            _, H, W = lst.shape
            anio = np.full((1, H, W), anio_file, dtype='float32')
            mes = np.full((1, H, W), mes_file, dtype='float32')
            dia = np.full((1, H, W), dia_file, dtype='float32')
            este, norte = grilla_xy(profile)
            este = este[np.newaxis, :, :]
            norte = norte[np.newaxis, :, :]

            capa = {
                'anio':anio, 'mes':mes, 'dia':dia, 'sea_water':sea_water, 'fresh_water':fresh_water, 'builds':builds, 'clouds':clouds, 'bare_ground':bare_ground, 'veg':veg, 
                'lst':lst, 'ndvi':ndvi, 'ndbi':ndbi, 'ndwi': ndwi, 'st_emissivity':st_emissivity, 'norte':norte, 'este':este,
            }

            arrays = [capa[f] for f in feature_order]
            multiband = np.concatenate((arrays), axis=0)
            num_bands, alto, ancho = multiband.shape

            data = multiband.reshape(num_bands, -1).T
            df_pix = pd.DataFrame(data, columns=feature_order)

            #profile_out = profile.copy()
            #profile_out.update(dtype=rasterio.uint8, count=1, height=int(alto), width=int(ancho))

            for name, model in best_models.items():
                preds = model.predict(df_pix)
                classification = preds.reshape(alto, ancho)

                out_path = os.path.join(folder_path, f"Temperature_{name}.TIF")

                profile.update(dtype=rasterio.float32, count=1, height=alto, width=ancho, nodata = np.nan)

                with rasterio.open(out_path, "w", **profile) as dst:
                    dst.write(classification.astype("float32"), 1)
                
                print("Procesado: ", out_path)

In [10]:
path = '/Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena'
clasificar_rasters(path,best_models)

Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena/LC08_L2SP_009053_20230117_20230131_02_T1/Heat_Islands_P5LinearRegression.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena/LC08_L2SP_009053_20230117_20230131_02_T1/Heat_Islands_P5RandomForestRegressor.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena/LC08_L2SP_009053_20230117_20230131_02_T1/Heat_Islands_P5ExtraTrees.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena/LC08_L2SP_009053_20230117_20230131_02_T1/Heat_Islands_P5GradientBoostingRegressor.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena/LC08_L2SP_009053_20230117_20230131_02_T1/Heat_Islands_P5SVR.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena/LC08_L2SP_009053_20230117_20230131_02_T1/Heat_Islands_P5MLPRegressor.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/cartagena/LC08_L2SP_009053_20210212_202